In [ ]:
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import os
import time

genai.configure(api_key="")

model = genai.GenerativeModel("gemini-1.5-pro")

In [3]:
import os
#import openai
import psycopg2 #for database connection
import pandas as pd 
import sqlalchemy
import numpy as np
import matplotlib.pyplot as plt # for plotting
import json
import re

# Kruskal-Wallis analysis of variance
import scipy.stats as ss # For Kruskal-Wallis test
import scikit_posthocs as sp #For post hoc tests. 
from termcolor import colored # for coloring the print text
import warnings # to ignore plot warnings

# For ggplot
from plotnine import *

# For bolding the printed text
from termcolor import colored

In [3]:
# Define database
hostname = 'localhost'
# f = open("Username.txt", "r")
# username = f.read()
# password = f.read()
database = 'phishdatabase'
username = 'postgres'
password = 'postgres'
port = "5432"

In [4]:
# Create Connection
try:
    connection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database, port=port )
 
except:
    print("I am unable to connect to the database")

In [5]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 10)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [ ]:
#Getting Image Urls
sql = """select c.subject_id as subject_id, json_extract_path_text(locations::json, '0') as image_url
        from zooniverse_phish_subjects_beta as c;
"""

image_url = pd.read_sql_query(sql, connection)
unique_beta_subject_ids = image_url['subject_id'].unique() 
image_url


In [ ]:
#Getting Image Urls
sql = """select c.subject_id as subject_id, json_extract_path_text(locations::json, '0') as image_url
        from zooniverse_phish_subjects_phase1 as c;
"""

image_url = pd.read_sql_query(sql, connection)
unique_phase1_subject_ids = image_url['subject_id'].unique() 
image_url

In [ ]:
sql = "select subject_id, subject_id_ph1 from cybertrust_zooniverse_datamatch;"

gold_standard = pd.read_sql_query(sql, connection)
unique_gold_subject_ids = gold_standard['subject_id'].unique() 
gold_standard


In [ ]:
gold_standard =gold_standard.dropna(subset=["subject_id_ph1"])
unique_gold_subject_ids = gold_standard['subject_id'].unique() 
gold_standard
gold_standard =gold_standard.dropna(subset=["subject_id"])
gold_standard

In [ ]:

#Getting Image Malicious Values

sql = """select subject_id, malicious
    from cybertrust_zooniverse_datamatch
"""

maliciousness = pd.read_sql_query(sql, connection)

###unique_subject_ids = maliciousness['subject_id'].unique() 
unique_datamatch_subject_ids = maliciousness['subject_id'].unique() 
maliciousness
# Changing Subject_id from varying to int
change_type ={"subject_id": "int64"}
image_url = image_url.astype(change_type)
data_types = image_url.dtypes
data_types

# Merging the tables
match = pd.merge(image_url, maliciousness, on="subject_id")
unique_match_subject_ids = match['subject_id'].unique() 
match  

# Merging the tables
match = pd.merge(gold_standard, match, on="subject_id")
unique_match_subject_ids = match['subject_id'].unique() 
match  


In [18]:
import urllib.request

def download_image(url, save_as):
    urllib.request.urlretrieve(url, save_as)

ans = pd.DataFrame(columns= ['url', 'subject_id', 'maliciousness', 'response'])
ans

,url,subject_id,maliciousness,response


In [ ]:
for x in range(1925,2000):
    #time.sleep(30)
    url = match["image_url"][x]
    subject_id = match["subject_id_ph1"][x]
    malicious = match["malicious"][x]
    path = f"./images/{subject_id}.jpg"
    download_image(url, path)
    sample_file = genai.upload_file(path= path,
                            display_name="Jetpack drawing")
    response = model.generate_content([sample_file, """ Generate as Output as json with the following categories: Findings as string, Present Cue Types as a list, Website Description as a string, TrustWorthiness Rating as a string, Visual Attractiveness Rating as a string
                                Please determine whether you think the image of the website or Email is Phishing Image or Not.
                               If you believe the image is Phishy say "Something's Phishy", if not say "Nothing Phishy Here". 
                               Findings category should hold "Something's Phishy" or "Nothing Phishy Here,"  
                               If you believe the image is Phishy, then tell me what Phishy cues do you think are present in this list:
                               - Invalid Domain or Sender
                               - Poor Spelling or Grammar
                               - Appeal to Action - Greed 
                               - Appeal to Action - Urgency 
                               - Appeal to Action - Authority
                               - Potentially Malicious Link
                               - Other Phishy Findings
                               Then, tell me your opinion regarding the design and flow of the image. Using a five point Likert Scale, tell me how trustworthy you think the image:
                                - "Very Trustworthy"
                                - "Trustworthy"
                                - "Neutral" 
                                - "Untrustworthy" 
                                - "Very Untrustworthy".
                               What I am looking for here is for you to rate the level of trust you have that the contents of the e-mail or web page are real. Second, I want you to tell me how pleasing or attractive the website or e-mail looks.
                               In this case, your answer can range on a five-point Likert scale:
                                - “Very Visually Attractive or Appealing” 
                                - "Visually Attractive or Appealing"
                                - "Neutral"
                                - "Visually Unattractive or Unappealing"
                                - “Very Visually Unattractive or Unappealing”. 
                                If you don't like how an image looks, tell me. """], 
                                safety_settings={
                                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        })
    
    data = {
        "url": url,
        "subject_id": subject_id,
        "maliciousness": malicious,
        "response": response.text
    }

    ans.loc[len(ans)] = data
    print(x)
    #time.sleep(15)
    


In [16]:
for x in range(0, len(ans)):
    print(x)
    print(ans['response'][x])

0
Findings: Nothing Phishy Here
Present Cue Types: []
Website Description: A screenshot of the English Wikipedia article on art.
TrustWorthiness Rating: Very Trustworthy
Visual Attractiveness Rating: Neutral 



In [22]:
ans.to_csv("Google_raw_output.csv", index=False)

Loading Data into dataframe

In [4]:
raw_output = pd.read_csv("Google_raw_output.csv")
raw_output

,url,subject_id,maliciousness,response
0,https://panoptes-uploads.zooniverse.org/produc...,52837250.0,True,Findings: Nothing Phishy Here\nPresent Cue Typ...
1,https://panoptes-uploads.zooniverse.org/produc...,52837251.0,True,"{\n ""Findings"": ""Something's Phishy"",\n ""Prese..."
2,https://panoptes-uploads.zooniverse.org/produc...,52837253.0,True,"{\n ""Findings"": ""Something's Phishy"",\n ""Pre..."
3,https://panoptes-uploads.zooniverse.org/produc...,52837254.0,True,Findings: Nothing Phishy Here\nPresent Cue Typ...
4,https://panoptes-uploads.zooniverse.org/produc...,52837255.0,True,"{\n ""Findings"": ""Something's Phishy"",\n ""Prese..."
...,...,...,...,...
1920,https://panoptes-uploads.zooniverse.org/produc...,52847471.0,False,Findings: Nothing Phishy Here\nPresent Cue Typ...
1921,https://panoptes-uploads.zooniverse.org/produc...,52847473.0,False,"{\n ""Findings"": ""Nothing Phishy Here"",\n ""Pres..."
1922,https://panoptes-uploads.zooniverse.org/produc...,52847474.0,False,Findings: Nothing Phishy Here\nPresent Cue Typ...
1923,https://panoptes-uploads.zooniverse.org/produc...,52847476.0,False,Findings: Nothing Phishy Here\nPresent Cue Typ...


In [ ]:
for x in range(1,10):
    print(f"{x}:")
    print(raw_output['response'][x])

In [88]:
sorted = pd.DataFrame(columns= ['subject_id', 'url', 'maliciousness', 'Findings','Present Cue Types', 'Website Description', 'TrustWorthiness Rating', 'Visual Attractiveness Rating','response'])
sorted

,subject_id,url,maliciousness,Findings,Present Cue Types,Website Description,TrustWorthiness Rating,Visual Attractiveness Rating,response


In [89]:
raw_output

,url,subject_id,maliciousness,response
0,https://panoptes-uploads.zooniverse.org/produc...,52837250.0,True,Findings: Nothing Phishy Here\nPresent Cue Typ...
1,https://panoptes-uploads.zooniverse.org/produc...,52837251.0,True,"{\n ""Findings"": ""Something's Phishy"",\n ""Prese..."
2,https://panoptes-uploads.zooniverse.org/produc...,52837253.0,True,"{\n ""Findings"": ""Something's Phishy"",\n ""Pre..."
3,https://panoptes-uploads.zooniverse.org/produc...,52837254.0,True,Findings: Nothing Phishy Here\nPresent Cue Typ...
4,https://panoptes-uploads.zooniverse.org/produc...,52837255.0,True,"{\n ""Findings"": ""Something's Phishy"",\n ""Prese..."
...,...,...,...,...
1920,https://panoptes-uploads.zooniverse.org/produc...,52847471.0,False,Findings: Nothing Phishy Here\nPresent Cue Typ...
1921,https://panoptes-uploads.zooniverse.org/produc...,52847473.0,False,"{\n ""Findings"": ""Nothing Phishy Here"",\n ""Pres..."
1922,https://panoptes-uploads.zooniverse.org/produc...,52847474.0,False,Findings: Nothing Phishy Here\nPresent Cue Typ...
1923,https://panoptes-uploads.zooniverse.org/produc...,52847476.0,False,Findings: Nothing Phishy Here\nPresent Cue Typ...


In [90]:
output = raw_output['response'][5]

print(output)

data = {}
for line in output.split('\n'):
    # Check if line contains ':' and is not empty
    if ': ' in line:
        key, value = line.split(': ', 1)  # Split at the first occurrence of ': '
        data[key] = eval(value) if value == '[]' else value  # Evaluate empty list

data

Findings: Something's Phishy
Present Cue Types: ['Invalid Domain or Sender', 'Appeal to Action - Authority', 'Other Phishy Findings']
Website Description: The image shows a fake Adobe PDF online document viewer that asks for the user to confirm their identity by signing in with their email address and password. This is a common phishing tactic used to steal credentials.
TrustWorthiness Rating: 'Very Untrustworthy'
Visual Attractiveness Rating: 'Visually Unattractive or Unappealing'



{'Findings': "Something's Phishy",
 'Present Cue Types': "['Invalid Domain or Sender', 'Appeal to Action - Authority', 'Other Phishy Findings']",
 'Website Description': 'The image shows a fake Adobe PDF online document viewer that asks for the user to confirm their identity by signing in with their email address and password. This is a common phishing tactic used to steal credentials.',
 'TrustWorthiness Rating': "'Very Untrustworthy'",
 'Visual Attractiveness Rating': "'Visually Unattractive or Unappealing'"}

In [61]:

raw_output_info = {'url': raw_output['url'][x],
        'subject_id': raw_output['subject_id'][x],
        'maliciousness': raw_output['maliciousness'][x],
        'response': raw_output['response'][x]
        }

data.update(raw_output_info)
sorted.loc[len(sorted)] = data

#0 3 and 5


In [ ]:
for x in range(0,1950):
    output = raw_output['response'][x]
    
    if output.strip().startswith('{'):
        response = json.loads(raw_output['response'][x])
    else:
        response = {}
        for line in output.split('\n'):
            # Check if line contains ':' and is not empty
            if ': ' in line:
                key, value = line.split(': ', 1)  # Split at the first occurrence of ': '
                response[key] = eval(value) if value == '[]' else value  # Evaluate empty list


    raw_output_info = {'url': raw_output['url'][x],
        'subject_id': raw_output['subject_id'][x],
        'maliciousness': raw_output['maliciousness'][x],
        'response': raw_output['response'][x]
        }
    
    response.update(raw_output_info)
    sorted.loc[len(sorted)] = response

    print(x)

In [11]:
sorted

,subject_id,url,maliciousness,Findings,Present Cue Types,Website Description,TrustWorthiness Rating,Visual Attractiveness Rating,response
0,52837250.0,https://panoptes-uploads.zooniverse.org/produc...,True,Nothing Phishy Here,[],The image is a screenshot of the Wikipedia pag...,Very Trustworthy,Neutral,Findings: Nothing Phishy Here\nPresent Cue Typ...
1,52837251.0,https://panoptes-uploads.zooniverse.org/produc...,True,Something's Phishy,"[Invalid Domain or Sender, Appeal to Action - ...",The website has a black background with a ligh...,Very Untrustworthy,Visually Unattractive or Unappealing,"{\n ""Findings"": ""Something's Phishy"",\n ""Prese..."
2,52837253.0,https://panoptes-uploads.zooniverse.org/produc...,True,Something's Phishy,"[Invalid Domain or Sender, Potentially Malicio...",The image is a screenshot of a website that ap...,Very Untrustworthy,Visually Unattractive or Unappealing,"{\n ""Findings"": ""Something's Phishy"",\n ""Pre..."
3,52837254.0,https://panoptes-uploads.zooniverse.org/produc...,True,Nothing Phishy Here,[],The image shows a screenshot of a website call...,Neutral,Visually Unattractive or Unappealing,Findings: Nothing Phishy Here\nPresent Cue Typ...
4,52837255.0,https://panoptes-uploads.zooniverse.org/produc...,True,Something's Phishy,"[Invalid Domain or Sender, Potentially Malicio...",This webpage appears to be a LinkedIn login pa...,Very Untrustworthy,Neutral,"{\n ""Findings"": ""Something's Phishy"",\n ""Prese..."
...,...,...,...,...,...,...,...,...,...
1920,52847471.0,https://panoptes-uploads.zooniverse.org/produc...,False,Nothing Phishy Here,[],A website about women's clothing and lingerie....,Neutral,Visually Unattractive or Unappealing,Findings: Nothing Phishy Here\nPresent Cue Typ...
1921,52847473.0,https://panoptes-uploads.zooniverse.org/produc...,False,Nothing Phishy Here,[],This webpage appears to be a general website d...,Neutral,Visually Unattractive or Unappealing,"{\n ""Findings"": ""Nothing Phishy Here"",\n ""Pres..."
1922,52847474.0,https://panoptes-uploads.zooniverse.org/produc...,False,Nothing Phishy Here,[],The image shows the homepage of a website call...,Trustworthy,Visually Attractive or Appealing,Findings: Nothing Phishy Here\nPresent Cue Typ...
1923,52847476.0,https://panoptes-uploads.zooniverse.org/produc...,False,Nothing Phishy Here,[],This webpage promotes a blog called Clueless F...,Neutral,Visually Unattractive or Unappealing,Findings: Nothing Phishy Here\nPresent Cue Typ...


In [62]:
# Finding all empty entries in the "Findings" column
empty_findings = sorted[sorted['Findings'].isna() | (sorted['Findings'] == '')]
empty_findings


,subject_id,url,maliciousness,Findings,Present Cue Types,Website Description,TrustWorthiness Rating,Visual Attractiveness Rating,response
1009,52843198.0,https://panoptes-uploads.zooniverse.org/produc...,False,NaN,NaN,NaN,NaN,NaN,Findings:Nothing Phishy Here\nPresent Cue Type...


In [64]:
print(raw_output['response'][1009])

Findings:Nothing Phishy Here
Present Cue Types:[]
Website Description:The image shows the landing page for individual marketplace health insurance through Premera Blue Cross. It features a mother and daughter and highlights open enrollment for a January 1 start date. 
TrustWorthiness Rating:Very Trustworthy
Visual Attractiveness Rating:Visually Attractive or Appealing



In [68]:
sorted.iloc[1009]

subject_id                                                             52843198.0
url                             https://panoptes-uploads.zooniverse.org/produc...
maliciousness                                                               False
Findings                                                      Nothing Phishy Here
Present Cue Types                                                              []
Website Description             The image shows the landing page for individua...
TrustWorthiness Rating                                           Very Trustworthy
Visual Attractiveness Rating                     Visually Attractive or Appealing
response                        Findings:Nothing Phishy Here\nPresent Cue Type...
Name: 1009, dtype: object

In [3]:
import pandas as pd

sorted = pd.read_csv("Google_sorted.csv")

print(sorted["Findings"].unique())

hit_count = ((sorted["maliciousness"] == True) & (sorted['Findings'] == "Something's Phishy")).sum()
Miss = ((sorted["maliciousness"] == True) & (sorted['Findings'] == "Nothing Phishy Here")).sum()
False_Alarm = ((sorted["maliciousness"] == False) & (sorted['Findings'] == "Something's Phishy")).sum()
Correct_Rejection = ((sorted["maliciousness"] == False) & (sorted['Findings'] == "Nothing Phishy Here")).sum()

print(f"""Hit: {hit_count}
       Miss: {Miss}
       False_Alarm: {False_Alarm}
        Correct_Rejection: {Correct_Rejection}""")


['Nothing Phishy Here' "Something's Phishy"]
Hit: 468
       Miss: 368
       False_Alarm: 27
        Correct_Rejection: 1062


In [94]:
#sorted.to_csv("Google_sorted.csv", index=False)

In [5]:
print(sorted["Visual Attractiveness Rating"].unique())

['Neutral' 'Visually Unattractive or Unappealing'
 'Very Visually Unattractive or Unappealing'
 'Visually Attractive or Appealing ' 'Untrustworthy' 'Untrustworthy '
 'Visually Attractive or Appealing' 'Trustworthy' '**Neutral** '
 "'Very Visually Unattractive or Unappealing'"
 'Very Visually Unattractive or Unappealing ' 'Visually Appealing '
 'Visually Attractive or Appealing. The colors and theme are coherent. ']
